In [1]:
from htm.bindings.sdr import SDR, Metrics
from htm.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood
import numpy as np
import pandas as pd
import pathlib
import datetime
import csv
from datetime import datetime
import os
from htm.encoders.rdse import RDSE, RDSE_Parameters
import time
import traceback
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn_extra.cluster import KMedoids
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from random import shuffle


In [2]:
inputSources = [
   "monthly_sp500_pca.csv",
   "weekly_dow_jones.csv",
   "weekly_nasdaq.csv",
   "weekly_sp500.csv",
   "monthly_vix_close.csv",
   "monthly_vix_high.csv",
   "monthly_vix_low.csv",
   "monthly_vix_open.csv",
   "daily_natural_gas.csv",
   "daily_oil_prices.csv",
   "value1_vix_close.csv",
   "value1_vix_high.csv",
   "value1_vix_low.csv",
   "value1_vix_open.csv",
   "monthly_gold_prices.csv"
]

In [ ]:

config = {
    'enc': {
        "value" :
            {'resolution': 0.88, 'size': 700, 'sparsity': 0.02},
        "time": 
            {'timeOfDay': (30, 1), 'weekend': 21}
    },
    'sp': {
        'inputDimensions': None,
        'columnDimensions': (1638,),
        'potentialPct': 0.85,
        'potentialRadius': None,
        'globalInhibition': True,
        'localAreaDensity': 0.04395604395604396,
        'synPermInactiveDec': 0.006,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'boostStrength': 3.0,
        'wrapAround': True,
        'seed': 1,
        'learn': False,
    },
    'tm': {
        'cellsPerColumn': 13,
        'activationThreshold': 17,
        'initialPermanence': 0.21,
        'minThreshold': 10,
        'maxNewSynapseCount': 32,
        'permanenceIncrement': 0.1,
        'permanenceDecrement': 0.1,
        'predictedSegmentDecrement': 0.0,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'learn': True
    },
    'anomaly': {'period': 1000},
    'learnRows': 100,
    'reflexSize': 2048,
    'accuracyThreshold': 0.5,
    'controlThreshold': 4
}


In [4]:
class ReflexiveMemory:
  def __init__(self, reflexSize, dimensions_dense, dimensions_sparse_sp):
    self.acKey0 = None
    self.pairs = {}
    self.tableSize  = reflexSize
    self.dimensions_dense = dimensions_dense
    self.dimensions_sparse_sp = dimensions_sparse_sp

  def add(self, denseColumns):
    acKey1 = '-'.join(map(str, denseColumns.sparse))
    if(self.acKey0 != None):

      sequence = self.pairs.get(self.acKey0, {})
      sequence_data = sequence.get(acKey1, {
         "count": 0,
         "time": datetime.now()
      })
      if sequence_data["count"] < 256:
        sequence_data["count"] = sequence_data["count"] + 1
      sequence_data["time"] = datetime.now()

      if self.pairs.get(self.acKey0, None) is None:
        self.pairs[self.acKey0] = { acKey1: sequence_data }
      else:
        self.pairs[self.acKey0][acKey1] = sequence_data
        
      table_entries = 0
      oldKey1 = None
      oldKey2 = None
      oldTime = datetime.now()
      for key1, value1 in self.pairs.items():
        table_entries = table_entries + len(value1.items())
        for key2, value2 in value1.items():
          if value2['time'] < oldTime:
            oldKey1 = key1
            oldKey2 = key2
            oldTime = value2['time']
      if table_entries > self.tableSize:
        del self.pairs[oldKey1][oldKey2]
        if len(self.pairs[oldKey1].items()) == 0:
          del self.pairs[oldKey1]

    self.acKey0 = acKey1

  def predict(self, denseColumns):
    return_count = 0
    return_sdr = None

    acKey = '-'.join(map(str, denseColumns.sparse))
    sequences = self.pairs.get(acKey, {})
    for sequence_key, sequence_data in sequences.items():
      if sequence_data["count"] > return_count:
        return_count = sequence_data["count"]
        return_sdr = sequence_key

    if return_sdr is not None:
      tmp_sdr = SDR( self.dimensions_dense )
      tmp_sdr.sparse = list(map(int, return_sdr.split('-')))
      return_sdr = tmp_sdr
    else:
      return_count = None

    return return_count, return_sdr


In [ ]:
class ControlUnit:
  def __init__(self, controlThreshold):
    self.anomalyRM = []
    self.anomalyTM = []
    self.anomalyNU = []
    self.anomalyCU = []
    self.historyRM = []
    self.historyTM = []
    self.historyGT = []
    self.historyCU = []
    self.countRMCU = 0
    self.controlThreshold = controlThreshold

  def anomalyScore(self, y, x):
      if np.count_nonzero(y) != 0:
          return 1 - np.count_nonzero((x & y)) / np.count_nonzero(y)
      return 1

  def compute(self, denseColumns1, sp, tm, rm):

    if rm.acKey0 is not None:

      denseColumns0 = SDR( rm.dimensions_dense )
      denseColumns0.sparse = list(map(int, rm.acKey0.split('-')))

      tm.activateDendrites(True)
      predictiveCells = tm.getPredictiveCells()

      predictiveColumns = SDR( rm.dimensions_sparse_sp )
      predictiveColumns.sparse = list(set(sorted(list(np.where(predictiveCells.dense == 1)[0]))))

      reflexiveColumns = SDR( rm.dimensions_sparse_sp )
      reflexiveCount, denseReflexiveColumns = rm.predict(denseColumns0)
      if denseReflexiveColumns is not None:
        sp.compute(denseReflexiveColumns, False, reflexiveColumns)

      activeColumns0 = SDR( rm.dimensions_sparse_sp )
      sp.compute(denseColumns0, False, activeColumns0)

      activeColumns1 = SDR( rm.dimensions_sparse_sp )
      sp.compute(denseColumns1, False, activeColumns1)

      self.historyRM.append( reflexiveColumns.dense )
      self.historyTM.append( predictiveColumns.dense )
      self.historyGT.append( activeColumns1.dense )

      self.anomalyNU.append(tm.anomaly)
      self.anomalyRM.append( self.anomalyScore(activeColumns1.dense, reflexiveColumns.dense) )
      self.anomalyTM.append( self.anomalyScore(activeColumns1.dense, predictiveColumns.dense) )

      if (len(self.anomalyRM) > self.controlThreshold) and (sum(self.anomalyRM[(-1-self.controlThreshold):-1]) > sum(self.anomalyTM[-1-self.controlThreshold:-1])):
        self.anomalyCU.append( self.anomalyTM[-1] )
        self.historyCU.append( self.historyTM[-1] )
        self.countRMCU = self.countRMCU + 1
      else:
        self.anomalyCU.append( self.anomalyRM[-1] )
        self.historyCU.append( self.historyRM[-1] )


In [6]:
input_path = pathlib.Path('../datasets/numenta')
dataset_metrics = []

pbar = tqdm(total=len(inputSources))
for dataset in inputSources:

    tm_infer_tm = 0
    tm_infer_rm = 0

    records = []
    with open(input_path.joinpath(dataset), "r") as fin:
        reader = csv.reader(fin)
        headers = next(reader)
        next(reader)
        next(reader)
        for record in reader:
            records.append(record)
        
    scalarEncoderParams = RDSE_Parameters()
    scalarEncoderParams.size = config["enc"]["value"]["size"]
    scalarEncoderParams.sparsity = config["enc"]["value"]["sparsity"]
    scalarEncoderParams.resolution = config["enc"]["value"]["resolution"]
    scalarEncoder = RDSE( scalarEncoderParams )
    encodingWidth = (scalarEncoder.size)

    config['sp']['inputDimensions'] = (encodingWidth,)
    config['sp']['potentialRadius'] = encodingWidth

    sp = SpatialPooler(
        inputDimensions = config['sp']['inputDimensions'],
        columnDimensions = config['sp']['columnDimensions'],
        potentialPct = config['sp']['potentialPct'],
        potentialRadius = config['sp']['potentialRadius'],
        globalInhibition = config['sp']['globalInhibition'],
        localAreaDensity = config['sp']['localAreaDensity'],
        synPermInactiveDec = config['sp']['synPermInactiveDec'],
        synPermActiveInc = config['sp']['synPermActiveInc'],
        synPermConnected = config['sp']['synPermConnected'],
        boostStrength = config['sp']['boostStrength'],
        wrapAround = config['sp']['wrapAround'],
        seed = config['sp']['seed']
    )

    tm = TemporalMemory(
        columnDimensions = config['sp']['columnDimensions'],
        cellsPerColumn = config['tm']['cellsPerColumn'],
        activationThreshold = config['tm']['activationThreshold'],
        initialPermanence = config['tm']['initialPermanence'],
        connectedPermanence = config['sp']['synPermConnected'],
        minThreshold = config['tm']['minThreshold'],
        maxNewSynapseCount = config['tm']['maxNewSynapseCount'],
        permanenceIncrement = config['tm']['permanenceIncrement'],
        permanenceDecrement = config['tm']['permanenceDecrement'],
        predictedSegmentDecrement = config['tm']['predictedSegmentDecrement'],
        maxSegmentsPerCell = config['tm']['maxSegmentsPerCell'],
        maxSynapsesPerSegment = config['tm']['maxSynapsesPerSegment']
    )

    rm = ReflexiveMemory( config['reflexSize'] , config['sp']['inputDimensions'], config['sp']['columnDimensions'])
    cu = ControlUnit( config['controlThreshold'] )

    try:
        
        for count, record in enumerate(records):
            
            learn_sp = config['sp']['learn']
            learn_tm = config['tm']['learn']
            if count < config['learnRows']:
                learn_sp = True
                learn_tm = True

            consumption = float(record[1])
            consumptionBits = scalarEncoder.encode(consumption)
            encoding = SDR( consumptionBits )

            cu.compute(encoding, sp, tm, rm)

            rm.add(encoding)

            activeColumns = SDR( sp.getColumnDimensions() )

            tmp_tm = time.time()
            sp.compute(encoding, learn_sp, activeColumns)
            tm.compute(activeColumns, learn=learn_tm)
            tm_infer_tm = tm_infer_tm + (time.time() - tmp_tm)

            tmp_tm = time.time()
            rm.predict(encoding)
            tm_infer_rm = tm_infer_rm + (time.time() - tmp_tm)

    except Exception as e:
        print(traceback.format_exc())
        print(e)

    def roc_auc_score_multiclass(y_true, y_pred):
        scores = []
        for y_class in set(y_true):
            y_true_class = [True if x == y_class else False for x in y_true]
            y_pred_class = [True if x == y_class else False for x in y_pred]
            scores.append(roc_auc_score(y_true_class, y_pred_class))
        return sum(scores) / len(scores)

    def match(y, x, idx1, cu, accuracyThreshold):
        n_samples = len(y)
        score1 = cu.anomalyScore(y[idx1], x[idx1])
        if score1 > (1 - accuracyThreshold):
            idx_closest = None
            score_closest = None
            for idx2 in range(n_samples):
                score2 = cu.anomalyScore(y[idx2], x[idx1])
                if score_closest is None or score_closest > score2:
                    score_closest = score2
                    idx_closest = idx2
            return idx_closest
        return idx1

    def calculateMetricsAnomaly(anomaly_scores, config, suffix):

        metric = {}

        anomaly_probability = []
        anomaly_history = AnomalyLikelihood(config["anomaly"]["period"])
        for anomaly_value in anomaly_scores:
            anomaly_probability.append( anomaly_history.compute(anomaly_value) )

        metric['anomaly-avg-'+suffix] = sum(anomaly_scores) / len(anomaly_scores)
        metric['anomaly-samples-'+suffix] = len(anomaly_scores)
        metric['anomaly-prob-avg-'+suffix] = np.count_nonzero(anomaly_probability) / len(anomaly_probability)

        return metric

    def calculateMetrics(Y_dataset, X_dataset, anomaly_scores, total_infe_time, config, cu, suffix):

        metric = {}

        n_samples = len(Y_dataset)

        Y_labels = list(range(n_samples))
        X_labels = [ match(Y_dataset, X_dataset, idx, cu, config['accuracyThreshold']) for idx in range(n_samples)]
        precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
        metric['total-infer-time-'+suffix] = total_infe_time
        metric['infer-time-'+suffix] = total_infe_time / len(anomaly_scores)
        metric['accuracy-'+suffix] = accuracy_score(Y_labels, X_labels)
        metric['precision-'+suffix] = precision
        metric['recall-'+suffix] = recall
        metric['fscore-'+suffix] = fscore
        metric['support-'+suffix] = support
        metric['auc-'+suffix] = roc_auc_score_multiclass(Y_labels, X_labels)

        metric.update( calculateMetricsAnomaly(anomaly_scores, config, suffix) )

        return metric
    
    tm_infer_cu = 0
    avg_infer_time_rm = tm_infer_rm / len(cu.anomalyRM)
    avg_infer_time_tm = tm_infer_tm / len(cu.anomalyTM)
    tm_infer_cu = tm_infer_cu + (avg_infer_time_rm * cu.countRMCU)
    tm_infer_cu = tm_infer_cu + (avg_infer_time_tm * (len(cu.anomalyCU) - cu.countRMCU))

    metric = {}
    metric['dataset'] = dataset
    metric['cu-rm-count'] = cu.countRMCU
    metric.update( calculateMetrics(cu.historyGT, cu.historyRM, cu.anomalyRM, tm_infer_rm, config, cu, 'rm') )
    metric.update( calculateMetrics(cu.historyGT, cu.historyTM, cu.anomalyTM, tm_infer_tm, config, cu, 'tm') )
    metric.update( calculateMetrics(cu.historyGT, cu.historyCU, cu.anomalyCU, tm_infer_cu, config, cu, 'cu') )
    metric.update( calculateMetricsAnomaly(cu.anomalyNU, config, 'nupic') )

    dataset_metrics.append(metric)
    pbar.update(1)
    # break

pbar.close()

100%|██████████| 15/15 [05:46<00:00, 23.13s/it]


In [7]:
df = pd.DataFrame(dataset_metrics)
df.to_csv('metrics.csv', index=False)

In [8]:
table_1_features = ['dataset','accuracy-cu','accuracy-rm','accuracy-tm','anomaly-avg-cu','anomaly-avg-rm','anomaly-avg-tm','anomaly-avg-nupic']
df[table_1_features]

,dataset,accuracy-cu,accuracy-rm,accuracy-tm,anomaly-avg-cu,anomaly-avg-rm,anomaly-avg-tm,anomaly-avg-nupic
0,monthly_sp500_pca.csv,0.403413,0.167581,0.409506,0.589656,0.805657,0.584941,0.584941
1,weekly_dow_jones.csv,0.037999,0.010101,0.039442,0.937517,0.984534,0.933455,0.933455
2,weekly_nasdaq.csv,0.211058,0.075000,0.212981,0.762213,0.906524,0.759923,0.759949
3,weekly_sp500.csv,0.348871,0.130226,0.351754,0.627456,0.844119,0.624673,0.624686
4,monthly_vix_close.csv,0.957778,0.718519,0.960988,0.066433,0.356289,0.059420,0.059667
5,monthly_vix_high.csv,0.943951,0.676543,0.948889,0.079595,0.354897,0.072027,0.072274
6,monthly_vix_low.csv,0.959753,0.722469,0.966914,0.058882,0.299825,0.049321,0.049564
7,monthly_vix_open.csv,0.944198,0.699259,0.950123,0.076540,0.358018,0.067373,0.067599
8,daily_natural_gas.csv,0.988099,0.969472,0.997758,0.019777,0.054983,0.006243,0.006415
9,daily_oil_prices.csv,0.988916,0.918193,0.992651,0.028583,0.207875,0.019285,0.019406


In [9]:
table_2_features = ['dataset','anomaly-prob-avg-cu','anomaly-prob-avg-rm','anomaly-prob-avg-tm','anomaly-prob-avg-nupic','anomaly-samples-cu','anomaly-samples-rm','anomaly-samples-tm','anomaly-samples-nupic']
df[table_2_features]

,dataset,anomaly-prob-avg-cu,anomaly-prob-avg-rm,anomaly-prob-avg-tm,anomaly-prob-avg-nupic,anomaly-samples-cu,anomaly-samples-rm,anomaly-samples-tm,anomaly-samples-nupic
0,monthly_sp500_pca.csv,0.391225,0.391225,0.391225,0.391225,1641,1641,1641,1641
1,weekly_dow_jones.csv,0.519481,0.519481,0.519481,0.519481,2079,2079,2079,2079
2,weekly_nasdaq.csv,0.519712,0.519712,0.519712,0.519712,2080,2080,2080,2080
3,weekly_sp500.csv,0.519942,0.519942,0.519942,0.519942,2081,2081,2081,2081
4,monthly_vix_close.csv,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
5,monthly_vix_high.csv,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
6,monthly_vix_low.csv,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
7,monthly_vix_open.csv,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
8,daily_natural_gas.csv,0.827699,0.827699,0.827699,0.827699,5798,5798,5798,5798
9,daily_oil_prices.csv,0.879639,0.879639,0.879639,0.879639,8300,8300,8300,8300


In [10]:
table_3_features = ['dataset','infer-time-cu','infer-time-rm','infer-time-tm','total-infer-time-cu','total-infer-time-tm','total-infer-time-rm']
df[table_3_features]

,dataset,infer-time-cu,infer-time-rm,infer-time-tm,total-infer-time-cu,total-infer-time-tm,total-infer-time-rm
0,monthly_sp500_pca.csv,0.000037,0.000010,0.000355,0.060210,0.582114,0.016804
1,weekly_dow_jones.csv,0.000102,0.000008,0.000339,0.212947,0.704953,0.017534
2,weekly_nasdaq.csv,0.000045,0.000009,0.000391,0.094483,0.813709,0.019660
3,weekly_sp500.csv,0.000023,0.000010,0.000347,0.047261,0.722818,0.019901
4,monthly_vix_close.csv,0.000025,0.000010,0.000299,0.102225,1.209260,0.041380
5,monthly_vix_high.csv,0.000031,0.000011,0.000366,0.123868,1.482902,0.043924
6,monthly_vix_low.csv,0.000028,0.000010,0.000244,0.113623,0.986581,0.039493
7,monthly_vix_open.csv,0.000032,0.000011,0.000354,0.129848,1.433515,0.044080
8,daily_natural_gas.csv,0.000044,0.000007,0.000064,0.257667,0.373704,0.039650
9,daily_oil_prices.csv,0.000023,0.000008,0.000119,0.195004,0.983771,0.068779


In [11]:
df['cu-infer-speedup'] = 1 - (df['total-infer-time-cu'] / df['total-infer-time-tm'])
df['cu-accuracy-improvement'] = df['accuracy-cu'] - df['accuracy-tm']
df['cu-tm-count'] = df['anomaly-samples-cu'] - df['cu-rm-count']

table_4_features = ['dataset','cu-accuracy-improvement','cu-infer-speedup','cu-rm-count','cu-tm-count','anomaly-samples-cu','infer-time-rm','infer-time-tm']
df[table_4_features]

,dataset,cu-accuracy-improvement,cu-infer-speedup,cu-rm-count,cu-tm-count,anomaly-samples-cu,infer-time-rm,infer-time-tm
0,monthly_sp500_pca.csv,-0.006094,0.896567,1515,126,1641,0.000010,0.000355
1,weekly_dow_jones.csv,-0.001443,0.697927,1488,591,2079,0.000008,0.000339
2,weekly_nasdaq.csv,-0.001923,0.883885,1884,196,2080,0.000009,0.000391
3,weekly_sp500.csv,-0.002883,0.934616,2000,81,2081,0.000010,0.000347
4,monthly_vix_close.csv,-0.003210,0.915465,3839,211,4050,0.000010,0.000299
5,monthly_vix_high.csv,-0.004938,0.916470,3825,225,4050,0.000011,0.000366
6,monthly_vix_low.csv,-0.007160,0.884832,3733,317,4050,0.000010,0.000244
7,monthly_vix_open.csv,-0.005926,0.909420,3800,250,4050,0.000011,0.000354
8,daily_natural_gas.csv,-0.009659,0.310506,2014,3784,5798,0.000007,0.000064
9,daily_oil_prices.csv,-0.003735,0.801779,7155,1145,8300,0.000008,0.000119


In [12]:
table_5_features = ['dataset'] + sorted(list(set(df.columns) - (set(table_1_features) | set(table_2_features) | set(table_3_features)  | set(table_4_features))))
df[table_5_features]

,dataset,auc-cu,auc-rm,auc-tm,fscore-cu,fscore-rm,fscore-tm,precision-cu,precision-rm,precision-tm,recall-cu,recall-rm,recall-tm,support-cu,support-rm,support-tm
0,monthly_sp500_pca.csv,0.701524,0.583537,0.704573,0.386609,0.161228,0.392806,0.380329,0.159652,0.386423,0.403413,0.167581,0.409506,None,None,None
1,weekly_dow_jones.csv,0.518768,0.504812,0.519490,0.034424,0.008851,0.035707,0.033165,0.008618,0.034368,0.037999,0.010101,0.039442,None,None,None
2,weekly_nasdaq.csv,0.605339,0.537278,0.606301,0.201188,0.071475,0.202983,0.197116,0.070406,0.198839,0.211058,0.075000,0.212981,None,None,None
3,weekly_sp500.csv,0.674279,0.564904,0.675721,0.334948,0.125774,0.338072,0.329411,0.124099,0.332615,0.348871,0.130226,0.351754,None,None,None
4,monthly_vix_close.csv,0.978884,0.859224,0.980489,0.954161,0.714677,0.957958,0.953060,0.714120,0.956926,0.957778,0.718519,0.960988,None,None,None
5,monthly_vix_high.csv,0.971968,0.838232,0.974438,0.940248,0.673590,0.945472,0.938991,0.673128,0.944279,0.943951,0.676543,0.948889,None,None,None
6,monthly_vix_low.csv,0.979872,0.861200,0.983453,0.956459,0.719340,0.964030,0.955166,0.718593,0.962853,0.959753,0.722469,0.966914,None,None,None
7,monthly_vix_open.csv,0.972092,0.849592,0.975056,0.940293,0.695025,0.946331,0.938983,0.694229,0.945061,0.944198,0.699259,0.950123,None,None,None
8,daily_natural_gas.csv,0.994049,0.984733,0.998879,0.987763,0.968788,0.997394,0.987707,0.968671,0.997262,0.988099,0.969472,0.997758,None,None,None
9,daily_oil_prices.csv,0.994457,0.959091,0.996325,0.987734,0.915377,0.991562,0.987321,0.914687,0.991158,0.988916,0.918193,0.992651,None,None,None
